In [1]:
%pylab inline
from pandas import Series, DataFrame
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》
作者: Wes McKinney
译者: 唐学韬

处理缺失数据

数据不完整在数据分析的过程中很常见。

pandas使用浮点值`NaN`表示浮点和非浮点数组里的缺失数据。

pandas使用`isnull()`和`notnull()`函数来判断缺失情况。

对于缺失数据一般处理方法为:

+ 滤掉, 使用 `dropna()`
+ 填充，使用 `fillna`，比如 ffill(指定值）或 bfill（插值

In [2]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
# None 也被认为是缺失
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

# 滤除缺失数据

In [7]:
from numpy import nan as NA
data = Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [8]:
# 使用 dropna()方法滤除
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [10]:
# 等价方法
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
# 对于 DataFrame, 会 drop 掉所有有 NA 的行
data = DataFrame([[1., 6.5, 3.], [1., NA, NA],
                  [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [15]:
#  只丢弃全为空的行
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [16]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [17]:
# 丢弃列
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [18]:
df = DataFrame(np.random.randn(7, 3))
df.ix[:4, 1] = NA; df.ix[:2, 2] = NA
df

,0,1,2
0,-1.890923,NaN,NaN
1,1.020294,NaN,NaN
2,-3.132870,NaN,NaN
3,0.541084,NaN,-0.203774
4,2.200062,NaN,1.775465
5,-0.458244,0.208688,-0.027016
6,-1.291057,0.391487,-1.341282


In [21]:
# thresh ：选取至少包含多少个非空值的行
df.dropna(thresh=2)

,0,1,2
3,0.541084,NaN,-0.203774
4,2.200062,NaN,1.775465
5,-0.458244,0.208688,-0.027016
6,-1.291057,0.391487,-1.341282


# 填充缺失数据

In [24]:
# 用指定值填充
df.fillna(0)

,0,1,2
0,-1.890923,0.000000,0.000000
1,1.020294,0.000000,0.000000
2,-3.132870,0.000000,0.000000
3,0.541084,0.000000,-0.203774
4,2.200062,0.000000,1.775465
5,-0.458244,0.208688,-0.027016
6,-1.291057,0.391487,-1.341282


In [26]:
# 每列用不同的值填充
df.fillna({1: 0.5, 3: -1})

,0,1,2
0,-1.890923,0.500000,NaN
1,1.020294,0.500000,NaN
2,-3.132870,0.500000,NaN
3,0.541084,0.500000,-0.203774
4,2.200062,0.500000,1.775465
5,-0.458244,0.208688,-0.027016
6,-1.291057,0.391487,-1.341282


In [27]:
# inplace: 不返回新对象，就地修改
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-1.890923,0.000000,0.000000
1,1.020294,0.000000,0.000000
2,-3.132870,0.000000,0.000000
3,0.541084,0.000000,-0.203774
4,2.200062,0.000000,1.775465
5,-0.458244,0.208688,-0.027016
6,-1.291057,0.391487,-1.341282


In [28]:
df = DataFrame(np.random.randn(6, 3))
df.ix[2:, 1] = NA; df.ix[4:, 2] = NA
df

,0,1,2
0,-0.201570,-0.301825,-0.292899
1,0.407622,0.154548,0.337993
2,-1.390216,NaN,1.952913
3,-0.155158,NaN,0.732646
4,-1.391272,NaN,NaN
5,-0.155741,NaN,NaN


In [30]:
# 用前值填充
df.fillna(method='ffill')

,0,1,2
0,-0.201570,-0.301825,-0.292899
1,0.407622,0.154548,0.337993
2,-1.390216,0.154548,1.952913
3,-0.155158,0.154548,0.732646
4,-1.391272,0.154548,0.732646
5,-0.155741,0.154548,0.732646


In [32]:
#  用前值，最多填充2次
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.201570,-0.301825,-0.292899
1,0.407622,0.154548,0.337993
2,-1.390216,0.154548,1.952913
3,-0.155158,0.154548,0.732646
4,-1.391272,NaN,0.732646
5,-0.155741,NaN,0.732646


In [34]:
# 用平均值填充
data = Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

fillna()参数汇总：

- value
- method, 插值方法，默认为 ffill
- axis, 
- inplace
- limit